# Biological Background

## Diauxisc growth in E. coli

Microorganisms constantly adjust their metabolism to the nutrients available around them. One of the most classic examples of this adaptive behavior is diauxic growth, a two-phase growth pattern observed when E. coli is provided with both glucose and acetate. E. coli strongly prefers glucose because it yields energy quickly and requires minimal enzyme investment. As long as glucose is available, the cell directs most of its metabolic machinery toward consuming it. When consuming glucose rapidly, E. coli produces acetate which is secreted in a process called **overflow metabolism.** Once glucose is depleted, this acetate can be reuptaken by the cell to be oxidized. This two step growth process is called **diauxic growth.** In between these two growth phases, the cell must downregulate it's glucose metabolizing mechanisms and upregulate its acetate metabolizing mechanisms, leading to a short lag phase. 

### Respiration, Fermentation and Acetate metabolism
In E.coli, three interconnected modes of metabolism - respiration, fermentation, and acetate metabolism - are central to the dynamics observed during this form of diauxic growth. These three modes differ strongly in ATP yield, efficiency and proteome cost, and together they account for the characteristic pattern of glucose depletion, acetate excretion and acetate reuse present in diauxic growth. 

**Respiration: high yield, oxygen dependent ATP generation** 
Under aerobic conditions, E. coli uses respiration which fully oxidizes glucose to $CO_2$ and yields ~10x more ATP per glucose than fermentation. Respiration supports rapid biomass production, has little overflow metabolites and is proteome intensive as it requires many enzyme complexes. 

**Fermentation: low yield, high rate overflow metabolism**
When oxygen is limited or the respiratory proteome becomes saturated, E. coli switches to fermentation and secretes partially oxidized byproducts, predominantly acetate. This mode bypasses the TCA cycle resulting in low ATP yield but higher rates. This means ATP can be produced quickly but inefficiently and at the cost of acetate overflow. 

**Acetate Metabolism**
Once glucose has been depleted and acetate is the only available carbon source, E. coli can switch to acetate consumption and convert acetate back into acetyl-CoA to be fed into the high yield TCA cycle. This results in slower gowth than the glucose phase as carbon uptake rates decrease even as higher ATP yields are seen than in fermentation. 



# Mathematical Background
## dynamic Flux Balance Analysis

While intracellular fluxes are steady-state at each moment, the environment changes over time. In diauxic growth, the switching period from glucose to acetate consumption is a dynamic process inherently not in steady state. This breaks one of the assumptions of FBA, so Mahadevan et al. developed the Stochastic Optimization Approach to enable modeling of this approach with dynamic FBA (dFBA). This approach wraps the inner, steady-state FBA optimization in time steps where available nutrient concentrations are updated at each time step according to the growth rate and consumption previously calculated. 

## 1. The Metabolic Layer (Inner Problem)

Inside each cell, thousands of reactions convert nutrients → biomass.

The model represents these reactions using:

### Stoichiometric Matrix

$$S \cdot v = 0$$

Where:
- $v$ = vector of reaction rates (fluxes)
- $S$ = stoichiometric matrix (network structure showing how metabolites connect)

This equation enforces **mass conservation**: no metabolite can magically appear or disappear.

### Optimization Objective

Cells must choose one flux distribution among many. FBA assumes they choose the one maximizing growth:

$$\max \mu(v)$$

subject to:

$$S \cdot v = 0, \quad v_{lb} \leq v \leq v_{ub}$$

**Interpretation:**
The cell picks the fastest-growing steady-state metabolic strategy allowed by physics and nutrients.

### Enzyme-Constrained FBA (ecFBA)

To model proteome limitations, we add:

$$\sum_{i} \frac{v_i \cdot MW_i}{k_{cat,i}} \leq E_{total}$$

**Meaning:**
- Each reaction uses part of a limited "enzyme budget"
- Slow enzymes (low $k_{cat}$) cost more
- Fast enzymes cost less

This explains overflow metabolism naturally.




## 2. The Dynamic Layer (Outer Problem)

### Biomass Update

Biomass grows exponentially with growth rate $\mu(t)$:

$$X(t + \Delta t) = X(t) \cdot e^{\mu(t) \Delta t}$$

### Substrate Updates

Extracellular glucose (Glc) and acetate (Ac) change due to consumption/secretion:

$$G(t + \Delta t) = G(t) + v_{glc}(t) \cdot X(t) \cdot \Delta t$$

$$A(t + \Delta t) = A(t) + v_{ac}(t) \cdot X(t) \cdot \Delta t$$

Where:
- **Negative flux** = uptake
- **Positive flux** = secretion


### Acetate Feedback Inhibition

Karlsen et al. observed that acetate secretion slows when extracellular acetate increases.

This can be modeled as:

$$v_{ac,max}(A) = \begin{cases} 0, & A > K_{fb} \\ \frac{-1}{\ln(K_{fb}/V_{max})} \cdot \ln(A/K_{fb}), & A \leq K_{fb} \end{cases}$$


In [1]:
import pickle
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, Markdown, HTML
import pandas as pd

In [2]:
exp_data = pd.read_csv('data/exp_data.csv')

with open('data/cond_histories.pkl', 'rb') as f:
    cond_histories = pickle.load(f)

with open('data/o2_sweep_results.pkl', 'rb') as f:
    o2_sweep_results = pickle.load(f)

In [3]:
def plot_exp_vs_sim(history, exp_data):
    """
    Single-panel comparison of experimental and simulated
    glucose, acetate, and biomass.
    """
    fig, ax1 = plt.subplots(figsize=(6,4))

    # Left axis: glucose + acetate
    line1, = ax1.plot(exp_data['Time'], exp_data['Glucose'], 'bo',
                      label='Glucose (exp)')
    line2, = ax1.plot(exp_data['Time'], exp_data['Acetate'], 'rs',
                      label='Acetate (exp)')
    line3, = ax1.plot(history['t'], history['glc'], 'b-',
                      label='Glucose (sim)')
    line4, = ax1.plot(history['t'], history['ac'], 'r-',
                      label='Acetate (sim)')

    ax1.set_xlabel('Time (h)')
    ax1.set_ylabel('Concentration (mM)')

    # Right axis: biomass / DW
    ax2 = ax1.twinx()
    line5, = ax2.plot(exp_data['Time'], exp_data['DW'], 'g^',
                      label='DW (exp)')
    line6, = ax2.plot(history['t'], history['X'], 'g-',
                      label='Biomass (sim)')
    ax2.set_ylabel('Dry Weight / Biomass (g/L)')

    # Shared legend
    lines = [line1, line3, line2, line4, line5, line6]
    labels = [l.get_label() for l in lines]
    fig.legend(lines, labels, loc='upper center',
               bbox_to_anchor=(0.5, 1.08), ncol=3, frameon=False)

    plt.tight_layout()
    plt.show()


def plot_exp_vs_sim_grid(cond_histories, exp_data):
    """
    2×2 grid showing the four conditions:
    - standard FBA ± feedback
    - ecFBA ± feedback
    using the precomputed conditional_histories dict.
    """
    fig, axes = plt.subplots(2, 2, figsize=(8, 6))
    axes_flat = axes.flat

    legend_lines = []
    legend_labels = []

    for idx, ((title, history), ax1) in enumerate(zip(cond_histories.items(), axes_flat)):
        # Left axis: glucose & acetate
        line1, = ax1.plot(exp_data['Time'], exp_data['Glucose'], 'bo', label='Glucose (exp)')
        line2, = ax1.plot(exp_data['Time'], exp_data['Acetate'], 'rs', label='Acetate (exp)')
        line3, = ax1.plot(history['t'], history['glc'], 'b-', label='Glucose (sim)')
        line4, = ax1.plot(history['t'], history['ac'], 'r-', label='Acetate (sim)')

        # Right axis: biomass
        ax2 = ax1.twinx()
        line5, = ax2.plot(exp_data['Time'], exp_data['DW'], 'g^', label='DW (exp)')
        line6, = ax2.plot(history['t'], history['X'], 'g-', label='Biomass (sim)')

        ax1.set_title(title)
        ax1.set_xlabel('Time (h)')
        ax1.set_ylabel('Concentration (mM)')
        ax2.set_ylabel('Dry Weight / Biomass (g/L)')

        if idx == 0:
            legend_lines = [line1, line3, line2, line4, line5, line6]
            legend_labels = [l.get_label() for l in legend_lines]

    # One shared legend under the grid
    fig.legend(legend_lines, legend_labels, loc='lower center',
               bbox_to_anchor=(0.5, 0), ncol=6, frameon=False)

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.12)
    plt.show()


In [4]:
cond_names = list(cond_histories.keys())

@interact(
    condition=widgets.Dropdown(
        options = cond_names,
        value=cond_names[0],
        description="Condition"
    )
)
def explore_condition(condition):
    """
    Pick one of the four model variants (standard/ecFBA, ± acetate feedback)
    """
    hist = cond_histories[condition]
    plot_exp_vs_sim(hist, exp_data)

    

interactive(children=(Dropdown(description='Condition', options=('standard FBA without acetate feedback', 'sta…

### Exercise 1.

**Which model produces more biomass, standard FBA or ecFBA? Why**


**Answer**
Standard FBA predicts higher biomass. Because there is no proteome limitation, the cell can use arbitrarily high amounts of enzyme to run high-flux, high-yield pathways. In ecFBA, the total enzyme pool is limited, so teh model must trade off between different pathways and cannot push growth as high. This lowers the achievable growth rate and total biomass, but is more biologically realistic because real cells have finite protein resources. Because respiratory pathways require substantial protein mass, ecFBA often predicts slower growth than unconstrained FBA—even when oxygen is abundant—because the cell must allocate finite proteome resources among competing pathways.

### Exercise 2.

**How does acetate feedback affect the transition from glucose growth to acetate growth?**

**Answer** As extracellular acetate increases, the model restricts the max possible acetate excretion flux. This limits the system from generating unrealistically high extracellular acetate spikes. This spike is quickly consumed, so when glucose depletes, there is abruptly little carbon source available and growth sharply slows. Feedback spreads out the amount of acetate over a longer period giving a smoother transition to slower growth.

In [ ]:
@interact(
    o2_max = widgets.SelectionSlider(
        options = sorted(o2_sweep_results.keys()),
        value = 10,
        description = "Available O₂ (mmol/gDW/h)"
    )
)

def explore_o2_sweep(o2_max):
    """
    Choose an O2 uptake bound.
    - See how glucose, acetate, and biomass trajectories change.
    """
    history = o2_sweep_results[o2_max]
    plot_exp_vs_sim(history, exp_data)

interactive(children=(SelectionSlider(description='Available O₂ (mmol/gDW/h)', index=9, options=(1, 2, 3, 4, 5…

### Exercise 3

**What conditions is the above slider for oxygen levels running?**

**Answer:** The simulation in teh oxygen slider is running ecFBA with acetate feedback. The growth rate doesn't reach higher than experimental levels due to proteome constraints, and the acetate doesn't peak indicating a feedback mechanism.

### Exercise 4.

**As $O_2$ availability increases, what happens to the time when glucose is depleted?**


**Answer:** increasing $O_2$ increases the cells ability to use respiration which generates more ATP per molecule of glucose compared to fermentation. So, at low $O_2$ levels:
- the cell relies more on fermentative pathways which are low-yield
- Growth is slower because fermentation provides less ATP per glucose. 
- Glucose is consumed more slowly

as $O_2$ levels increase:
- ATP yield per glucose increases
- growth rate increase which increases glucose uptake rate (more biomass -> more uptake)
- As a result, glucose is consumed faster

### Exercise 5

**Describe the behavior of growth as $O_2$ levels increase. Why does this happen?**

**Answer:** Growth doesn't increase linearly wiht $O_2$, instead it is slow at low levels, faster at moderate levels, and drops again at high levels. At lower levels of $O_2$, fermentation dominates which is less efficient and so growth is slow. As levels increase, respiration takes over which is more efficient and results in higher growth. After a point though, oxygen becomes abundant and the system hits other limits such as proteome allocation limits or substrate depletion timing. 

### Exercise 6

**For each feature in the above widgets, (FBA type, acetate feedback, $O_2$ level), describe what goes wrong in the model behavior if you remove (or decrease it).**

**Answer:**

Removing ecFBA:
- the model predicts unrealistically high growth rates as cells are "allowed" unlimited enzymes
- acetate overflow become extreme because FBA allows arbitrarily large fluxes. 
- Biomass curves overshoot experimentla values dramitically

Removing Acetate Feedback:
- acetate spikes sharply and unrealistically high early on in the simulation
- post glucose growth on acetate is abrupt as because acetate availability is unrealistically inflated

Reducing $O_2$ availability
- growth slows significantly
- fermentation becomes favored to max growth even though it is less efficient because it cannot run respiration at as high of a rate. 

Increasing $O_2$ avaialbitlity (too much)
- the model burns glucose too quickly leaving lesss time for growth on glucose
